In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../mnist/data/", one_hot=True)

Extracting ../mnist/data/train-images-idx3-ubyte.gz
Extracting ../mnist/data/train-labels-idx1-ubyte.gz
Extracting ../mnist/data/t10k-images-idx3-ubyte.gz
Extracting ../mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [3]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        
        _, cost_val = sess.run([optimizer, cost], 
                               feed_dict={X: batch_xs, Y: batch_ys})
        
        total_cost += cost_val
        
    print("Epoch:", "{:04d}".format(epoch+1), 
          "Avg. Cost:, " "{:.3f}".format(total_cost/total_batch))
    
print("Optimization completed!")

Epoch:    1 Avg. Cost:, 0.505
Epoch:    2 Avg. Cost:, 0.238
Epoch:    3 Avg. Cost:, 0.175
Epoch:    4 Avg. Cost:, 0.149
Epoch:    5 Avg. Cost:, 0.139
Epoch:    6 Avg. Cost:, 0.127
Epoch:    7 Avg. Cost:, 0.119
Epoch:    8 Avg. Cost:, 0.110
Epoch:    9 Avg. Cost:, 0.109
Epoch:   10 Avg. Cost:, 0.099
Epoch:   11 Avg. Cost:, 0.097
Epoch:   12 Avg. Cost:, 0.094
Epoch:   13 Avg. Cost:, 0.094
Epoch:   14 Avg. Cost:, 0.089
Epoch:   15 Avg. Cost:, 0.088
Epoch:   16 Avg. Cost:, 0.079
Epoch:   17 Avg. Cost:, 0.079
Epoch:   18 Avg. Cost:, 0.079
Epoch:   19 Avg. Cost:, 0.072
Epoch:   20 Avg. Cost:, 0.081
Epoch:   21 Avg. Cost:, 0.078
Epoch:   22 Avg. Cost:, 0.068
Epoch:   23 Avg. Cost:, 0.073
Epoch:   24 Avg. Cost:, 0.064
Epoch:   25 Avg. Cost:, 0.069
Epoch:   26 Avg. Cost:, 0.072
Epoch:   27 Avg. Cost:, 0.069
Epoch:   28 Avg. Cost:, 0.066
Epoch:   29 Avg. Cost:, 0.060
Epoch:   30 Avg. Cost:, 0.057
Optimization completed!


In [6]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print("Accuracy:", sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

Accuracy: 0.9709
